# Обучаем новую модель

### Устанавливаем библиотеки

In [1]:
!pip install transformers sentencepiece datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.6 MB/s eta 0:00:00


### Загружаем датасеты для обучения модели

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

train_dataset = pd.read_csv('/content/drive/MyDrive/Dataset/train.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')
val_dataset = pd.read_csv('/content/drive/MyDrive/Dataset/validation.csv')

train_dataset.shape[0], test_dataset.shape[0], val_dataset.shape[0]

(26945, 3367, 3294)

In [4]:
# Преобразуем строки в списки
train_dataset.labels = train_dataset.labels.apply(lambda x: eval(x))
test_dataset.labels = test_dataset.labels.apply(lambda x: eval(x))
val_dataset.labels = val_dataset.labels.apply(lambda x: eval(x))

In [5]:
LABELS = ['радость', 'интерес', 'удивление', 'печаль', 'гнев', 'отвращение', 'страх', 'вина', 'нейтрально']

# Выведим по 3 примера для каждой эмоции
for i in range(len(LABELS)):
    print(f"\nEMOTION: {LABELS[i]}")
    for t in train_dataset[train_dataset.labels.apply(lambda x: x==[i])].text.sample(3):
        print(t)


EMOTION: радость
Давайте победим этих Лейкерс!
Это лицо в 0:18. Это лицо счастья
С днём рождения и днём торта! Я надеюсь, что вы найдете немного радости где-нибудь в вашем сегодняшнем путешествии.

EMOTION: интерес
Почему все настаивают на попытке сделать это? Просто открой чертову бутылку [ИМЯ] большим пальцем, как приличный человек.
Как это расизм?
Это разрушает честь, правда??

EMOTION: удивление
Я до сих пор не могу поверить, что [ИМЯ] действительно будет в Smash.
Я удивлен, что не видел больше таких. Спасибо за поддержку :)
Это новый уровень блондинки, о котором я даже не подозревала!

EMOTION: печаль
Меня чуть не стошнило.
Я желаю ему удачи в попытках вернуть видео, но его шансы довольно малы. Один человек против корпоративной империи.
Дерьмо, которого она не хочет = «кураторская коробка красоты» 🤔

EMOTION: гнев
Человеческий стыд и культурное угнетение в отношении сексуального поведения несут ответственность за все беды мира.
Вот что в этом такого глупого. У моей собаки иногда 

In [6]:
# Преобразование индексов в бинарный вид
def binarize_labels(labels):
    return [float(i in labels) for i in range(9)]

print(binarize_labels([0]))
print(binarize_labels([1]))
print(binarize_labels([1, 4]))
print(binarize_labels([8]))

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


### Загружаем модель

In [7]:
from transformers import BertForSequenceClassification, AutoTokenizer

base_model = 'cointegrated/rubert-tiny2'

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = BertForSequenceClassification.from_pretrained(base_model, num_labels=9, problem_type='multi_label_classification')

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [14]:
model.cuda() # Дать доступ модели к GPU

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

In [ ]:
model.config.label2id = {label: i for i, label in enumerate(LABELS)}
model.config.id2label = {i: label for i, label in enumerate(LABELS)}

In [ ]:
# Готовим датасеты для обучения
TrainDict = [{**tokenizer(train_dataset.text[i], truncation=True), 'label': binarize_labels(train_dataset.labels[i])} for i in range(train_dataset.shape[0])]
TestDict = [{**tokenizer(test_dataset.text[i], truncation=True), 'label': binarize_labels(test_dataset.labels[i])} for i in range(test_dataset.shape[0])]
ValDict = [{**tokenizer(val_dataset.text[i], truncation=True), 'label': binarize_labels(val_dataset.labels[i])} for i in range(val_dataset.shape[0])]

In [ ]:
import gc
import torch

# Чистка ненужных данных при обучении
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [ ]:
from tqdm.auto import tqdm, trange
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support
from IPython.display import display

# Проверить точность модели на тестовых данных
def evaluate_model(model, dev_dataloader, verbose=False, labels=None):
    facts, preds = predict_with_model(model, dev_dataloader)
    aucs = get_classification_report(facts, preds, labels)
    if verbose:
        print('aucs:', aucs, np.mean(aucs))
    return np.mean(aucs)

# Получить прогноз модели
def predict_with_model(model, dataloader):
    preds = []
    facts = []

    for batch in tqdm(dataloader):
        facts.append(batch.labels.cpu().numpy())
        batch = batch.to(model.device)
        with torch.no_grad():
            pr = model(input_ids=batch.input_ids, attention_mask=batch.attention_mask, token_type_ids=batch.token_type_ids)
        preds.append(torch.softmax(pr.logits, -1).cpu().numpy())
    facts = np.concatenate(facts)
    preds = np.concatenate(preds)
    return facts, preds

# Получить результаты работы модели
def get_classification_report(facts, preds, labels=None):
    print(facts.shape, preds.shape)
    aucs = [roc_auc_score(facts[:, i], preds[:, i]) for i in range(7)]
    return aucs

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer)
batch_size = 64

# Создаем тренировочные и тестовые (валидационные) данные для обучения модели
train_dataloader = DataLoader(
    TrainDict,
    batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)
dev_dataloader = DataLoader(
    ValDict,
    batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)

In [ ]:
# Тестируем точность модели
evaluate_model(model, dev_dataloader, verbose=True)

  0%|          | 0/52 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(3294, 9) (3294, 9)
aucs: [0.5307886366129209, 0.46549253725122, 0.49787893260834953, 0.518944536143582, 0.47972804239845623, 0.6318092825060462, 0.408428089377871] 0.5047242938426352


0.5047242938426352

### Обучаем модель

In [ ]:
# Инициализируем параметры обучения

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)

ewm_loss = 0

window = 500
cleanup_step = 100

In [ ]:
# Обучаем модель

for _ in range(8):
    model.train()
    cleanup()

    # Делаем 5 итераций и тестируем модель
    for epoch in trange(5):
        tq = tqdm(train_dataloader)

        for i, batch in enumerate(tq):
            # Вычисляем функцию потерь (loss)
            try:
                batch = batch.to(model.device)
                output = model(**batch)
                loss = output.loss
                loss.backward()
            except RuntimeError as e:
                print('error on step', i, e)
                loss = None
                cleanup()
                continue

            # Обновляем веса
            optimizer.step()
            optimizer.zero_grad()

            if i % cleanup_step == 0:
                cleanup()

            w = 1 / min(i+1, window)
            ewm_loss = ewm_loss * (1-w) + loss.item() * w
            tq.set_description(f'loss: {ewm_loss:4.4f}')

    # Тестируем работу модели
    model.eval()
    eval_loss = evaluate_model(model, dev_dataloader, verbose=True)
    print(f'epoch {epoch + 1}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss}\n\n')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

...

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/422 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

(3294, 9) (3294, 9)
aucs: [0.8945189658620935, 0.899541804002155, 0.8427960860673305, 0.81467998974351, 0.8076350100398856, 0.8554679916421252, 0.8997759882039358] 0.859202262223005
epoch 5, step 421: train loss: 0.1760  val auc: 0.859202262223005




### Тестируем модель

In [ ]:
# Загружаем тестовые данные
test_dataloader = DataLoader(
    TestDict,
    batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)

In [ ]:
# Тестируем модель на новых тестовых данных
model.eval()
eval_loss = evaluate_model(model, dev_dataloader, verbose=True)
test_loss = evaluate_model(model, test_dataloader, verbose=True)
model.train()
print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss} test auc: {test_loss}')

  0%|          | 0/52 [00:00<?, ?it/s]

(3294, 9) (3294, 9)
aucs: [0.8447876946105852, 0.802183751496666, 0.8325703858824106, 0.8043446891029234, 0.7714123555176313, 0.8311314391010348, 0.8759392899676742] 0.8231956579541323


  0%|          | 0/53 [00:00<?, ?it/s]

(3367, 9) (3367, 9)
aucs: [0.8645165941151027, 0.8129417892763735, 0.8248583099198445, 0.7839904220960019, 0.7477532068938381, 0.8632409551592433, 0.8953499388149844] 0.8275216023250556
epoch 4, step 421: train loss: 0.0550  val auc: 0.8231956579541323 test auc: 0.8275216023250556


In [ ]:
# Получим предсказания для каждого теста
model.eval()
facts, preds = predict_with_model(model, test_dataloader)

  0%|          | 0/53 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import f1_score

# Вычисляем f1-score для каждой из эмоций 
pd.DataFrame([
    {av: f1_score(facts[:, i], preds[:, i] > 0.5, average=av) for av in ['binary', 'micro', 'macro']}
    for i in range(9)
]).round(4)

,binary,micro,macro
0,0.5911,0.9400,0.7794
1,0.4948,0.8987,0.7193
2,0.4647,0.9617,0.7224
3,0.4528,0.9139,0.7030
4,0.4651,0.8476,0.6881
5,0.4103,0.9658,0.6963
6,0.6277,0.9849,0.8100
7,0.7438,0.9908,0.8696
8,0.7169,0.7054,0.7049


In [ ]:
# Вычисляем средний f1-score для всех эмоций
pd.DataFrame([
    {av: f1_score(facts[:, i], preds[:, i] > 0.5, average=av) for av in ['binary', 'micro', 'macro']}
    for i in range(6)
]).mean().round(4)

binary    0.4798
micro     0.9213
macro     0.7181
dtype: float64

### Сохраняем обученную модель

In [ ]:
model.save_pretrained('/content/drive/MyDrive/emotion_detection_model')
tokenizer.save_pretrained('/content/drive/MyDrive/emotion_detection_model')

('/content/drive/MyDrive/emotion_detection_model/tokenizer_config.json',
 '/content/drive/MyDrive/emotion_detection_model/special_tokens_map.json',
 '/content/drive/MyDrive/emotion_detection_model/vocab.txt',
 '/content/drive/MyDrive/emotion_detection_model/added_tokens.json',
 '/content/drive/MyDrive/emotion_detection_model/tokenizer.json')

# Загружаем нашу модель

In [10]:
!pip install transformers -q

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer

model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/emotion_detection_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/emotion_detection_model')

### Реализуем функцию прогноза

In [ ]:
LABELS = ['радость', 'интерес', 'удивление', 'печаль', 'гнев', 'отвращение', 'страх', 'вина', 'нейтрально']

# Предсказание эмоции в тексте
@torch.no_grad()
def predict_emotion(text):
  inputs = tokenizer(text, truncation=True, return_tensors='pt')
  inputs = inputs.to(model.device)

  outputs = model(**inputs)

  pred = torch.nn.functional.softmax(outputs.logits, dim=1)
  pred = pred.argmax(dim=1)

  return LABELS[pred[0]]

# Вероятностный прогноз эмоций в тексте
@torch.no_grad()
def predict_emotions(text):
    inputs = tokenizer(text, truncation=True, return_tensors='pt')
    inputs = inputs.to(model.device)

    outputs = model(**inputs)

    pred = torch.nn.functional.softmax(outputs.logits, dim=1)

    emotions_list = {}
    for i in range(len(pred[0].tolist())):
        emotions_list[LABELS[i]] = round(pred[0].tolist()[i], 4)
    return emotions_list

### Тестируем модель

In [ ]:
print(predict_emotion('Люблю цветы'))
print(predict_emotion('Ненавижу цветы'))
print(predict_emotion('Мне страшно'))
print(predict_emotion('Куда мне стоит сегодня сходить?'))

радость
гнев
страх
интерес


In [ ]:
print(predict_emotions('Люблю цветы'))
print(predict_emotions('Ненавижу цветы'))
print(predict_emotions('Мне страшно'))
print(predict_emotions('Куда мне стоит сегодня сходить?'))

{'радость': 0.9918, 'интерес': 0.0054, 'удивление': 0.0002, 'печаль': 0.0008, 'гнев': 0.0003, 'отвращение': 0.0001, 'страх': 0.0002, 'вина': 0.0002, 'нейтрально': 0.001}
{'радость': 0.0001, 'интерес': 0.0001, 'удивление': 0.0001, 'печаль': 0.0003, 'гнев': 0.9978, 'отвращение': 0.001, 'страх': 0.0, 'вина': 0.0001, 'нейтрально': 0.0006}
{'радость': 0.0011, 'интерес': 0.0037, 'удивление': 0.0014, 'печаль': 0.0018, 'гнев': 0.0003, 'отвращение': 0.0058, 'страх': 0.9823, 'вина': 0.0009, 'нейтрально': 0.0026}
{'радость': 0.0001, 'интерес': 0.9932, 'удивление': 0.0002, 'печаль': 0.0002, 'гнев': 0.0004, 'отвращение': 0.0, 'страх': 0.0001, 'вина': 0.0001, 'нейтрально': 0.0058}
